## Notebook bla bla bla

What do I do in this notebook

In [ ]:
import pandas as pd
pd.options.display.max_columns = 50

In [ ]:
# Load in the data

nyc_accidents = pd.read_csv('data/nyc_bike_accidents.csv')
cph_accidents = pd.read_csv('data/cph_bike_accidents.csv')

### STEP 1: Combine the data

In [ ]:
# Rename the first column to make it easier to understand
cph_accidents.rename(columns={"Tilskadekomne og dræbte i færdselsuheld efter indblandede transportmidler, område, personskade og tid: Cykel": "injury_death"}, inplace=True)

# Get the data for 2021
cph_accidents_2021 = cph_accidents[['injury_death', "Area", "2021"]]

# Drop the last row which contains a note about the dataset, not actual data.
cph_accidents_2021.drop([6], inplace=True)

# Rename the columns to make it easier to understand and combine the two categories of injured cyclists (mild injury and serious injury) into one
cph_accidents_2021 = cph_accidents_2021.replace({'injury_death': {'Dræbte': 'NUMBER OF CYCLIST KILLED', 'Alvorligt tilskadekomne': 'NUMBER OF CYCLIST INJURED', 'Lettere tilskadekomne': 'NUMBER OF CYCLIST INJURED'}})

cph_accidents_2021 = cph_accidents_2021.groupby(['injury_death', 'Area']).sum().reset_index()


# And pivot to get Copenhagen and Frederiksberg in seperate columns. Ready to append the data from NYC.
cph_accidents_2021 = cph_accidents_2021.pivot(index='Area', columns='injury_death', values='2021').reset_index()
cph_accidents_2021

In [ ]:
# Since the latest accident in Copenhagen is from 2021, we will only look at accidents from 2021 in New York City.

nyc_accidents['CRASH DATE'] = pd.to_datetime(nyc_accidents['CRASH DATE'])

nyc_accidents['YEAR'] = nyc_accidents['CRASH DATE'].dt.strftime('%Y')

nyc_accidents = nyc_accidents.groupby('YEAR').sum('NUMBER OF PERSONS KILLED')
nyc_accidents = nyc_accidents[['NUMBER OF CYCLIST INJURED', 'NUMBER OF CYCLIST KILLED']].reset_index()
nyc_accidents

nyc_accidents = nyc_accidents[nyc_accidents['YEAR'].str.contains('2021')]
nyc_accidents['YEAR'].replace({'2021': 'New York City'}, inplace=True)

nyc_accidents.rename(columns={"YEAR": "Area"}, inplace=True)

nyc_accidents

In [243]:
accidents = cph_accidents_2021.append(nyc_accidents, ignore_index=True)

accidents

/var/folders/hx/m3n3wwr91yg_snnv68ynqsp80000gn/T/ipykernel_7355/1891637644.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  accidents = cph_accidents_2021.append(nyc_accidents, ignore_index=True)


,Area,NUMBER OF CYCLIST INJURED,NUMBER OF CYCLIST KILLED
0,Copenhagen,167.0,3.0
1,Frederiksberg,25.0,0.0
2,New York City,4961.0,19.0


### STEP 2: Standardize the data

I now have a dataframe with the number of injuries and fatalities in Copenhagen, Frederiksberg (a part of Copenhagen, will be merged with the rest of Copenhagen later) and New York City. I want to standardize the data so that I can compare the number of injuries and fatalities per share of the population that bikes. 

According to [NYC DOT](https://www.nyc.gov/html/dot/downloads/pdf/cycling-in-the-city-2021.pdf), 773,000 New Yorkers bike regularly (several times a month).

